## Step 0: Importing modules

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import yfinance as yf

## Step 1: Backtesting Example:

See week 3 steps 0-1 (close reading)


## Step 2: Sample Alpaca Code - Setup

Modified from https://github.com/alpacahq/alpaca-py/blob/master/examples/crypto/crypto-trading-basic.ipynb 

First we have to give Alpaca the necessary keys and endpoints to access our paper trading account. For testing's sake, we can just copy them in plaintext, but if you make/publish a public github repo with your strategy, **make sure these are never plaintext (use a config file or environment variables)!**

In [ ]:
# Paste your keys here
api_key = ""
secret_key = ""

paper = True # Please do not modify this. This example is for paper trading only.

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

We will use "alpaca-py" package in python to access the trading api. I suggest pip installing it, although it can be installed inline below:

In [ ]:
%pip install alpaca-py

In [ ]:
import alpaca

Here is some handy object definitions from the alpaca package we will use in our code:

In [ ]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.crypto import CryptoHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.crypto import CryptoDataStream

from alpaca.data.requests import (
    CryptoBarsRequest,
    CryptoQuoteRequest,
    CryptoTradesRequest,
    CryptoLatestQuoteRequest
    )
from alpaca.trading.requests import (
    GetAssetsRequest,
    MarketOrderRequest,
    LimitOrderRequest,
    StopLimitOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetClass,
    AssetStatus,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

## Step 3: Sample Alpaca Code - Using the Trading Client

First, let's set up a trading client variable that will be accessed to place orders:

In [ ]:
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

To test our access, we can try getting a json containing our account information (such as current cash)


In [ ]:
acct = trade_client.get_account()
acct


Now let's try executing a few paper trading orders (based on the real market) on our account from the code. We will use crypto since the markets for that are open 24/7 and our lectures are on Saturday.

Alpaca supports simple market orders, limit orders, stop limit orders, and different time-in-force combinations (see the docs for more info). Below, we will demonstrate an order buying 0.01 BTC at the market price, with order "Good Till Canceled".

Similar code for equities, options, etc can be found at: https://github.com/alpacahq/alpaca-py/tree/master/examples

In [ ]:

req = MarketOrderRequest(
    symbol = "BTC/USD",
    qty = 0.01,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
) #This defines the order

res = trade_client.submit_order(req) # this submits the order to the API, returning the result
res

Check your account dashboard. Notice any changes? If your code ran, you should see a new position in BTC/USD.

Selling/shorting, buying in $ amount instead of share amount, etc. can all be found in the docs and done similarly.

Now let's check our positions (we will see the ~800$ of BTC we are long):

In [ ]:
# get all open positions:
positions = trade_client.get_all_positions()
positions

Say we want to close the position. Placing another order to sell the BTC may leave us with some left over due to slippage, or not get fully filled. As such, it is easier to close our position in a given symbol as such (specifying quantity):

In [ ]:
trade_client.close_position(
    symbol_or_asset_id = "BTCUSD",
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

Now we can look at all our open positions again, and see BTC has been closed/sold:

In [ ]:
# get all open positions:
positions = trade_client.get_all_positions()
positions

## Step 4: Sample Alpaca Code - Getting Historical Market Data

As well as providing a paper-trading API, Alpaca provides an API for getting market data that uses the same symbol names it does for trading (making thing convenient). While the data isn't super high fidelity and lags live data by 5 minutes (IIRC), it is convenient for an academic example, and surely better for this purpose than something like yfinance.

Let's get crypto's historical data

In [ ]:
crypto_historical_data_client = CryptoHistoricalDataClient()
symbol="BTC/USD"

We can now get the historical 1-hour time bars for our symbol for the last day (this can be specified to whatever time frame you want, within reason). Be sure to specify your time zone!

The result returns a dataframe similar to what we are used to with yfinance.

In [ ]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/cryptobars-1
now = datetime.now(ZoneInfo("America/Chicago"))
req = CryptoBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
)
crypto_historical_data_client.get_crypto_bars(req).df

In the example code at https://github.com/alpacahq/alpaca-py/blob/master/examples/crypto/crypto-trading-basic.ipynb, you can also see how to get historical quotes and trades (although bars should suffice for basic strategies)

For our program to trade live, we will likely need it to repeatedly look at the latest quote for the symbol(s) it is trading on (L1 level market data). This can be done as such:

In [ ]:
req = CryptoLatestQuoteRequest(
    symbol_or_symbols = [symbol],
)
res = crypto_historical_data_client.get_crypto_latest_quote(req)
res

In your project, you will be required to use code like this to backtest and implement a live version of your algorithm. I highly suggest making custom functions using the ones above for simpler order execution!